In [1]:
"""
Dependency:
    data2.json
    live_data.py
"""
import numpy as np
import json
import time

## import snapshot data

In [2]:
# data2.json - Saurav's most updated data
with open('data2.json', 'r') as f:
    data = json.load(f)
data = data['data']['snapshots_new'][0]['snapshot_data']

In [3]:
# for quick access
addressList = list(data)
tokenList = list(data[addressList[0]])

# sanity check so that tokenList is universal
for addy in addressList:
    assert tokenList == list(data[addy]), 'wrong token list' 

print(f'number of addresses: {len(addressList)}')
print(f'number of pairs: {len(tokenList)}')

number of addresses: 456
number of pairs: 52


## import token price

In [4]:
ticker2id = {'ceth':'ethereum',
             'rowan':'sifchain',
             'cusdt':'tether',
             'cbat':'basic-attention-token', 
             'cant':'aragon',
             'cbnt':'bancor',
             'czrx':'0x',
             'clink':'chainlink',
             'cmana':'decentraland',
             'clrc':'loopring',
             'cenj':'enjincoin',
             'csnx':'havven',
             'ctusd':'true-usd', 
             'cocean':'ocean-protocol', 
             'cftm':'fantom',
             'csusd':'nusd',  
             'cusdc':'usd-coin',
             'ccro':'crypto-com-chain', 
             'cwbtc':'wrapped-bitcoin', 
             'csxp':'swipe',
             'cband':'band-protocol', 
             'cdai':'dai',
             'ccomp':'compound-governance-token',
             'cuma':'uma', 
             'cbal':'balancer',
             'cyfi':'yearn-finance',
             'csrm':'serum', 
             'ccream':'cream-2',
             'csand':'the-sandbox', 
             'csushi':'sushi',
             'cesd':'empty-set-dollar',
             'cuni':'uniswap',
             'caave':'aave',
             'cbond':'barnbridge',  
             'cwfil':'filecoin',
             'cgrt':'the-graph', 
             'clon':'tokenlon', 
             'c1inch':'1inch', 
             'crune':'thorchain', 
             'cwscrt':'secret',
             'ciotx':'iotex', 
             'creef':'reef-finance',
             'ccocos':'cocos-bcx',    
             'ckeep':'keep-network',
             'cogn':'origin-protocol',
             'cdaofi':'daofi',
             'clina':'lina',
             'ctshp':'12ships',
             'cb20':'b20',
             'cakro':'akropolis',
             'crfuel':'rio-defi',
             'crly':'rally-2'}

assert tokenList == list(ticker2id), 'wrong ticker2id'

In [5]:
def get_price(id_: str):
    import requests
    r = requests.get('https://api.coingecko.com/api/v3/simple/price',
                             params={'ids': id_,
                                     'vs_currencies': 'usd', 
                                     'include_24hr_change': 'false'})
    return r.json()[id_]['usd']

In [6]:
# fetch current token price from coingecko
price = {}
for token in tokenList:
    price[token] = get_price(ticker2id[token])
    print(token, price[token])
    time.sleep(.5) # time buffer

ceth 1686.79
rowan 0.977129
cusdt 1.0
cbat 1.07
cant 7.15
cbnt 7.42
czrx 1.55
clink 27.13
cmana 0.93744
clrc 0.523806
cenj 2.47
csnx 17.54
ctusd 0.997032
cocean 1.42
cftm 0.388052
csusd 1.0
cusdc 0.998003
ccro 0.23276
cwbtc 56007
csxp 3.26
cband 13.34
cdai 1.0
ccomp 376.95
cuma 22.94
cbal 52.79
cyfi 33810
csrm 4.89
ccream 118.83
csand 0.651806
csushi 16.37
cesd 0.149975
cuni 30.74
caave 337.3
cbond 52.05
cwfil 90.94
cgrt 1.52
clon 6.87
c1inch 4.18
crune 5.2
cwscrt 2.82
ciotx 0.04063985
creef 0.03740748
ccocos 1.36
ckeep 0.58791
cogn 0.87972
cdaofi 0.748602
clina 0.00726336
ctshp 0.02769286
cb20 8.46
cakro 0.054203
crfuel 0.168284
crly 0.847672


## process snapshots

In [7]:
# data = {address: { 'ceth': [0, 0, 0], 'crly': [100, 0, -100] }}
# [100, 100, 0]

snapshot = {} # 
for addy in addressList: # for each addy
    snapshotOfOneAddress = 0
    for token in tokenList: # aggregate USD values across all tokens
        snapshotOfOneAddress += np.cumsum(data[addy][token]) * price[token] # convert number of token to USD value, then cumsum
    snapshot[addy] = snapshotOfOneAddress

In [20]:
# an example
snapshot['sif106sm2envlp4l9haeg8njwjjwuuzn5nxjkjdu7c']

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [8]:
invalidAddressList = []

for addy in addressList:
    min_ = min(snapshot[addy])
    if min_ < 0: # contains negative snapshots
        max_ = max(snapshot[addy])
        print(addy)
        print(f'{max_:.2f}, {min_:.2f}, {-min_/max_:.2f}')
        print()
        invalidAddressList.append(addy)

sif10prgaf6ap3hkn8qtnzkys8vxm8ut0rr7jwzrqx
2715.39, -31.58, 0.01

sif10twgpy3un8w98l0qafj9900ennxhn7smgn03c3
0.32, -0.00, 0.00

sif123877lsx5d8ggmrsaz33canp50mmp6303kuah0
0.03, -0.14, 4.18

sif127yz2y9suh0v59kg32sw388nlvjkv39nxp4zds
622.65, -1.27, 0.00

sif12h557fex4a4ddgyxhs6jnuju94vwfsd54xv3h2
27556.74, -61663.36, 2.24

sif12zhwz792d8zpxj3wmz05c7k9meea6q0xf5mj3w
664.95, -230.19, 0.35

sif13dalre059t3u79pdmdwjw7c3peysrkwpj3knme
11.85, -155.10, 13.09

sif13hjd7kjd3t4lzvszrrtvyqevd9qrv9t0vf26ed
487.55, -725.02, 1.49

sif145uc8knqwh0xk4ssymleeuhny80ellwnsfyy3v
0.00, -0.25, inf

sif160k3s4fa8svgqpj4np07x0pfyt4c96p5uylr6v
413.36, -6237.44, 15.09

sif18t688mk59r5rgkjq3er0ag5q549grz3k286vy2
0.25, -1.86, 7.51

sif18u9r87wn0rlu954hq8w0vguq3jvdc3wxts83k2
24.01, -173200.13, 7213.19

sif18w7en9ld858zcrdedssmdv8e3ex8gz430h5daa
3530.31, -832.68, 0.24

sif19hrpkfqtak4sq9l6xgflcj3u8a754a5fh2n37t
0.30, -5.52, 18.58

sif1azflsehkgd5psf0kxhds2ggen039w6m6545mfn
0.09, -4.85, 53.57

sif1d39gkggjkvpt5e975fa

/Users/edenau/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  


In [9]:
print(len(invalidAddressList))

58


In [10]:
def inspect_address(data, address):
    d = data[address]
    tokenList = list(d)
    i = 0
    while len(d[tokenList[0]]) > i:
        for token in tokenList:
            if d[token][i] != 0:
                print(i, token, d[token][i])
        i += 1
    print()

In [11]:
inspect_address(data,
                address='sif1zx6secyr0gampkk7vna6l57walw0ps9jcplyq5')

60 cuni 0.3433824361810572
63 cuni -1.0361125110211078
91 cant 0.05461344524798463
91 cwbtc -6.510843093101398
197 cant -0.4883351893724547



In [23]:
# in the future, turn all negative numbers to 0
for addy, l in snapshot.items():
    snapshot[addy] = [max(0,e) for e in l]

In [25]:
# using the example above, no more negative numbers!
print(snapshot['sif1zx6secyr0gampkk7vna6l57walw0ps9jcplyq5'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10.555576088205697, 10.555576088205697, 10.555576088205697, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
# but for now, we ignore addresses that contain any negative snapshot
good_snapshot = snapshot.copy()
for addy in invalidAddressList:
    del good_snapshot[addy]

In [13]:
good_snapshot

{'sif1003l39amhtmlmh6eupj3w94xg6ljh5zz37dy8j': array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
  

## call functions

In [14]:
# Only use functions from utils blindly
from live_data import get_APY, get_geyser_accmulated_reward, elementwisesum, dict2list

# set constants
constants = {'miningSeconds':4*30*86400,
             'totalReward':30e6,
             'epochSeconds':10*60}

rowanPrice = price['rowan']

In [15]:
list_userSnapshots = dict2list(good_snapshot)
globalSnapshots = elementwisesum(list_userSnapshots)

In [16]:
apy = get_APY(totalStaked=globalSnapshots[-1], 
              rowanPrice=rowanPrice,
              **constants)

print(f'APY: {apy:.2f}%')

APY: 549.41%


In [17]:
countAccReward = 0
for userSnapshots, user in zip(list_userSnapshots, list(good_snapshot)):
    userAccReward = get_geyser_accmulated_reward(userSnapshots=userSnapshots, 
                                                 list_userSnapshots=list_userSnapshots, 
                                                 **constants)
    print(f'{user}: {userAccReward:.2f} ROWAN')
    countAccReward += userAccReward

sif1003l39amhtmlmh6eupj3w94xg6ljh5zz37dy8j: 0.00 ROWAN
sif1048c2lagwe84sz0sk4ncy2myrzxykjes40hn62: 0.11 ROWAN
sif106sm2envlp4l9haeg8njwjjwuuzn5nxjkjdu7c: 1.09 ROWAN
sif10dv233h7s2p0preh7zqu8x873m4p27d3dntfqt: 0.00 ROWAN
sif10f743f8p3ry7w77cget3p0085v5w56sap95tyg: 0.00 ROWAN
sif10jatqfd88m8s2uhtdtdl3txtayjtzsve2klyhh: 3.96 ROWAN
sif10jj3th2p0zr2s9nfvpp3aqkyuzwsmj47gmekhs: 0.00 ROWAN
sif10kmkcxpl5qjkmlavfr605ywm7gn5cfw5ys3f2d: 0.23 ROWAN
sif10m4m2zhq4wvav3cvtrc3wgw8wmyu9lkx3g5euu: 20.81 ROWAN
sif10sjena7yg9fgx3h5jeyvqxavvd2gxm42lgw7q6: 0.52 ROWAN
sif10ss96p7mm0s738se6553crwjkqcpl4gr4nznym: 0.01 ROWAN
sif10swk365ug5rwh8kuq44senas4kg5x7cwmrczez: 0.51 ROWAN
sif10u9j54gmk75j3y27x9hrw372f6fsdgf3d055ku: 0.00 ROWAN
sif10vqfcfdwg006v2kglyzg9n9ntwkvvwst29lp6l: 0.04 ROWAN
sif12fjkfzpuuyupjqk2fcs6f6s3k346qnur9zkr35: 0.00 ROWAN
sif12gnsxyv3lqmmtv6rl30pek727qarrqqwzpfwx9: 0.59 ROWAN
sif12ksw08s9gdapy0ngrml8fm4tp3vyju5e43lc59: 0.00 ROWAN
sif12ndtrnr8d7wsy37886uzn8rntmu22atyyn8vr0: 0.02 ROWAN
sif12nldg

sif1eyqz88f6kvjtehsdhr5nm5z663dyuvhmkvpas7: 0.01 ROWAN
sif1f2hcq38z5ynwsln07n7x7k0xed7eq0t7gyvury: 0.00 ROWAN
sif1f5p48xjpkzg7rmttt8s3kg8cll8dc2uhjcfqfk: 0.00 ROWAN
sif1f7lnqd0yq0pfctlc9e8rrp8f9yl5uxa8q7f87k: 0.00 ROWAN
sif1fdcdek5hqc3hzwmrr0kv80twzu98j77rgezfsd: 1.07 ROWAN
sif1ffwpcufza2a27pjmzcmatq34rhyd372jts73xa: 0.00 ROWAN
sif1fnpqtvcydqyxsrdmx9vtxus5znwy4tf4xrre4c: 0.51 ROWAN
sif1frxphs06uq2nzxthlpkn3d8j3qw0p5xymuqvae: 0.00 ROWAN
sif1fu54gzzgeykcqdv5m8lm0ggz7wy7f52u70du9g: 51.51 ROWAN
sif1fy87a8ytqfmlav47c8kgcgrxkppaftwyfujece: 0.02 ROWAN
sif1fzmgr6gjkqdfmmym9cm43kt3nm0z9kw2nnxwl6: 5.73 ROWAN
sif1g2ty64k3w5qyx60fse9xu3get3enhw4nl8urjc: 0.04 ROWAN
sif1gdm04gxmsw3jzcyq7sa9cc4r0s4gjk28pgjga7: 8.04 ROWAN
sif1gg5v7pakhzp3xh2ucmamk0mspmc9d60qrkv45v: 0.00 ROWAN
sif1gm6rdjj5uyrwlk5akur3c4mf6ggxcgnv229cu8: 0.00 ROWAN
sif1gmktcjgh09j6hlypg5q7q73reuqs53lrmzxdvx: 5.68 ROWAN
sif1grp704fgdj3z4kdefwhfl9mu77z9f9c0j9a9ak: 0.01 ROWAN
sif1gthamjkq04d3zyeq5skhm92h99u0tyykltgajh: 141.29 ROWAN
sif1gtv

sif1t9xkfkgl4jcwtvdlnqvwc69kh9utrlez4ze4ru: 0.00 ROWAN
sif1tasrdx34as3wzszjpvlpp289akmkl4yl8r605k: 6.34 ROWAN
sif1te8t38m0tx7l9h6ky0pfs85xgkf28zsncheyyv: 37.00 ROWAN
sif1teuuyn4k5htmzus9elaa4ylggcchswrf6rwfv5: 0.00 ROWAN
sif1tf4zge7gehglyvrvesn5dttzxlzv3kzhumy7aw: 65.15 ROWAN
sif1th3ujscq5cghp3n2fzd8kjuk2hz972mds2y4d4: 1063.58 ROWAN
sif1th89uf0zpf7y8tantw6sl92dry4l0v8a20ssg3: 4720.89 ROWAN
sif1tks6g2zg9sxvdha28frhm7gjkawq8afe3h7l40: 0.00 ROWAN
sif1tnasd2ucd22jgl8ermkkhder6h8y3yftea29wu: 497.62 ROWAN
sif1tpr36xw5etr3fhjafrhyqtrq9wrgpqsr6hn8e5: 0.00 ROWAN
sif1tpwtwxtwjs923e9u2tgt4nxx363svfuhe4k2fy: 5.27 ROWAN
sif1tqs2wjukcx66v0dh0vuuyf8kssp6d52v82vk6q: 221.12 ROWAN
sif1tr9g25wfgn4ftgw2rl2q88u2uy9wf2z2uzvhqt: 0.85 ROWAN
sif1tsmdvsd3d8fclvvl4kq0ket6npkpq5dp7y33ee: 2.21 ROWAN
sif1tytdf0g796r74ngtap0p5gtf0wmxef6j9lt9sj: 0.00 ROWAN
sif1tzr0pp65c7p6dc2xungt8vjmm8kg7twxkw9z77: 0.67 ROWAN
sif1u06fm45m29kw4hd09tc35xjdqrrnus2cptlzn0: 1.17 ROWAN
sif1u47htj2n2yeccthtq6ks0p5p8fjug5p4wm9ram: 0.02 ROWA

In [18]:
print(f'Total reward to be distributed: {countAccReward:.2f} ROWAN')

idealAccReward = constants['totalReward']/constants['miningSeconds']*constants['epochSeconds']*len(list_userSnapshots[0])
print(f'Total reward available: {idealAccReward:.2f} ROWAN')

Total reward to be distributed: 470486.11 ROWAN
Total reward available: 470486.11 ROWAN
